# Imports

In [2]:
import os
import pandas as pd
import psycopg2

## Read using pandas

In [46]:
songfolder = './../data/song_data/'
outputfolder = './../stagingarea/'
def process_folder(path):
    dfs = []
    for dirpath, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".json"):
                dfs.append(pd.read_json(os.path.join(os.path.abspath(dirpath), file), lines=True))
    df = pd.concat(dfs).reset_index(drop=True)
    return df

In [40]:
def readsongdata(df):
    usecols = [
        'artist_id',
     'artist_latitude',
     'artist_location',
     'artist_longitude',
    'artist_name',
     'duration',
     'song_id',
     'title', 
        'year'
    ]
    df = df[usecols]
    return df

In [4]:
def connecttopostgre():
    conn = psycopg2.connect("host=pgs dbname=mydb user=myuser password={}".format(os.environ.get("POSTGRES_PASSWORD")))
    conn.autocommit = True
    print(conn.status)
    return conn
conn = connecttopostgre()
cur = conn.cursor()

1


In [55]:
songs = process_folder(songfolder)
songs = readsongdata(songs)
songs.to_csv('raw_songs.csv', index=False, sep='|', encoding='utf-8')

In [54]:
q_copysongs = '''
COPY staging_songs (
    artist_id,
 artist_latitude,
 artist_location,
 artist_longitude,
 artist_name,
 duration,
 song_id,
 title, 
 year)
FROM '/home/data/raw_songs.csv'
WITH CSV HEADER DELIMITER '|';
'''
cur.execute(q_copysongs)

UndefinedTable: relation "staging_songs" does not exist


In [18]:
cur.execute('SELECT COUNT(*) FROM staging_songs LIMIT 5;')
cur.fetchall()

[(0,)]

In [19]:
cur.execute('SELECT COUNT(*) FROM songs LIMIT 5;')
cur.fetchall()

[(71,)]

In [20]:
cur.execute('SELECT COUNT(*) FROM artists LIMIT 5;')
cur.fetchall()

[(69,)]

In [27]:
df = pd.read_csv('/stagingarea/events.csv', sep='|')
df.sample(10)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
7472,Danilo Montero,Logged In,Kinsley,F,28,Young,233.48200,paid,"Red Bluff, CA",PUT,NextSong,1.540465e+12,436,El es el Rey,200,1542379177796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_...",85.0
1030,Anna Tatangelo,Logged In,Jordan,F,0,Hicks,218.93179,free,"Salinas, CA",PUT,NextSong,1.540009e+12,552,Il Mio Amico,200,1542196566796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",37.0
4376,Harmonia,Logged In,Avery,F,55,Watkins,655.77751,paid,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.540872e+12,324,Sehr kosmisch,200,1542288215796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30.0
5494,Wordsworth,Logged In,Lily,F,49,Koch,253.15220,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Right Now (Produced by Ayatollah),200,1542848253796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15.0
338,Angus & Julia Stone,Logged In,Mohammad,M,32,Rodriguez,290.71628,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,812,What You Wanted,200,1542994949796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88.0
4775,Havok,Logged In,Noah,M,0,Chavez,227.68281,free,"Ogden-Clearfield, UT",PUT,NextSong,1.540971e+12,150,Ivory Tower,200,1541392598796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:32.0) G...,94.0
4145,Sugar Pie DeSanto,Logged In,Tegan,F,16,Levine,169.50812,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Soulful Dress,200,1543350815796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80.0
2383,Cage The Elephant,Logged In,Rylan,M,20,George,196.38812,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,Lotus,200,1543594323796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16.0
7575,HÃÂ©roes del Silencio,Logged In,Kate,F,27,Harrell,245.02812,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,633,Parasiempre,200,1542392251796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97.0
987,Coldplay,Logged In,Jacqueline,F,12,Lynch,268.38159,paid,"Atlanta-Sandy Springs-Roswell, GA",PUT,NextSong,1.540224e+12,559,Yellow,200,1542191429796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",29.0
